In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, KFold
from imblearn.over_sampling import (SMOTE, BorderlineSMOTE, SVMSMOTE, SMOTENC,
                                    KMeansSMOTE)


Using TensorFlow backend.


In [2]:
#importing dataset
train = pd.read_csv('astro_train.csv')
test = pd.read_csv('astro_test.csv')



In [3]:

y_train= train['class']
train = train.drop(['class'], axis= 1)

dataset = pd.concat([train, test])
dataset = dataset.drop(['id','skyVersion', 'run', 'rerun', 'camCol'], axis= 1)

dataset["err_g_log"] = np.log(dataset["err_g"])
dataset["err_i_log"] = np.log(dataset["err_i"])
dataset["err_r_cbrt"] = np.cbrt(dataset["err_r"])
dataset["err_u_log"] = np.log(dataset["err_u"])
dataset["err_z_log"] = np.log(dataset["err_z"])
dataset["dec_cbrt"] = np.cbrt(dataset["dec"])
dataset["photoz_u_log"] = np.log(dataset["photoz"])
dataset["obj_z_log"] = np.log(dataset["obj"])

dataset["#ra_cat"] = pd.cut(dataset["#ra"],
                           bins=[0,180,220,240,np.inf],
                           labels=[1,2,3,4])

#dataset["dec_cat"] = pd.cut(dataset["dec"],         bins=[-1.25,-1.0,-0.25,np.inf],                  labels=[1,2,3])

dataset["extinction_r_cat"] = pd.cut(dataset["extinction_r"],
                           bins=[0,0.10,0.15,np.inf],
                           labels=[1,2,3])

dataset["field_cat"] = pd.cut(dataset["field"],
                           bins=[0,300,500,550,np.inf],
                           labels=[1,2,3,4])

dataset = dataset.drop(['err_g',  'err_r','err_i', 'err_u', 'err_z','photoz','obj', '#ra', 'dec', 'extinction_r', 'field'], axis= 1)

cat_attr = ["#ra_cat","dec_cat","extinction_r_cat","field_cat"]
num_attr = [i for i in list(dataset.columns) if i not in cat_attr]

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([('std_scaler',MinMaxScaler())])

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

full_pipeline = ColumnTransformer([
             ("num", num_pipeline, num_attr)
             ])

transformed_dataset = full_pipeline.fit_transform(dataset)


train_prepared = transformed_dataset[:45000, :]
test_prepared = transformed_dataset[45000:, :]



In [4]:
#over-sampling data
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

#smote =  
#smote =   SMOTE()
#
#nr = NearMiss()
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import NearMiss 
smote = BorderlineSMOTE(random_state=0, kind='borderline-2')

x_res, y_res = smote.fit_sample(train_prepared, y_train)

In [5]:
#creating model using random forest classifier
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 600, random_state = 0)
classifier.fit(x_res, y_res)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=600,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [6]:
#predicting output
y_pre = classifier.predict(test_prepared)
#checking number of elements in each class
from collections import Counter
Counter(y_pre)

Counter({0: 5748, 1: 5450, 2: 3802})

In [7]:
#checking accuracy
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_prepared, y_train, test_size=0.2)

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X= X_train, y = y_train, cv = 3)
accuracies
#accuracies.std()

array([0.66775   , 0.67058333, 0.67516667])

In [8]:
#saving predictionsn to csv file
y = pd.DataFrame(y_pre)
y.to_csv('Nabo1.csv')